In [ ]:
#we count the cells in the matrix from top to bottom, from left to right
#when plotted the matrix is shown "upside-down", with the (0, 0) cell 
#corresponding to the origin

In [ ]:
#The cells below can be put outside the iterative algorithm
#This is the back-bone of the data structure

In [ ]:
import numpy as np
import pandas as pd

#define domain size

L_x = 20; #cm
L_y = 2; #cm

#define size of cell in the cube structured mesh

del_l = 0.1; #cm

#define number of cells

N_x = int(L_x/del_l);
N_y = int(L_y/del_l);

#define the velocity vectors according to staggered arangement of velocity wrt scalar CV

u_vel = np.zeros((1, (N_x + 1)*N_y));
v_vel = np.zeros((1, N_x*(N_y + 1)));

#U velocity guess
u_vel[0, :] = 2;
#V velocity guess is 0

In [ ]:
#define the pressure vector with its boundaries already included

pressure = np.zeros((1, N_x*N_y));

pres_1 = 1000; #pressure inlet
pres_2 = 500; #pressure inside

pressure[0, 0 : N_y] = pres_1;
pressure[0, (N_x - 1)*N_y : N_x*N_y] = pres_2;

#PRESSURE GUESS
#define a linear pressure variation between these two points
difference_pres = (pres_1 - pres_2)/(N_x-1)

pres_iter = pres_1
for a_x in range(1, N_x):
    L_bound = int(a_x*N_y)
    U_bound = int((a_x +1)*N_y);
    pres_iter = pres_iter - difference_pres/a_x;
    pressure[0, L_bound : U_bound] = pres_iter

In [ ]:
#in order for the CGS solver to work we must count only the active cells, not the cell number in the entire domain

In [ ]:
#define the PRESSURE CORRECTION solution matrix

pressure_sol_mat = np.zeros(((N_x - 2)*N_y*5, 4)); #active cells are all except for inlet and outlet cell

#define column 0
for a_x in range(1, N_x -1):
    for a_y in range(0, N_y):
        cell_P = int(a_x*N_y + a_y);
        index_location_1 = (cell_P-N_y)*5 ;
        index_location_2 = index_location_1 + 5; #for 2D
        pressure_sol_mat[index_location_1 : index_location_2, 0] = cell_P;
        
#define boundary cells list
#corners, inlet, outlet and top, bottom walls

pressure_corners = [N_y, 2*N_y - 1, (N_x-2)*N_y, (N_x-1)*N_y -1 ];
inlet_pressure_cells = np.arange(N_y + 1, 2*N_y - 1, 1);
outlet_pressure_cells = np.arange((N_x-2)*N_y + 1, (N_x-1)*N_y -1, 1);
bottom_pressure_wall = np.arange(2*N_y, (N_x-2)*N_y, N_y);
top_pressure_wall = np.arange(3*N_y -1, (N_x-1)*N_y - 1, N_y);

print(pressure_corners);
print(inlet_pressure_cells);
print(outlet_pressure_cells);
print(bottom_pressure_wall);
print(top_pressure_wall);

#define gross structure of column 1
for a_x in range(1, N_x -1):
    for a_y in range(0, N_y):
        cell_P = int(a_x*N_y + a_y);
        
        location_sol_mat = (cell_P-N_y)*5;
        S = location_sol_mat;
        W = S + 1;
        P = S + 2;
        E = S + 3;
        N = S + 4;
        
        cell_S = cell_P -1;
        cell_W = cell_P - N_y;
        cell_E = cell_P + N_y;
        cell_N = cell_P + 1;
        
        pressure_sol_mat[S, 3] = cell_S;
        pressure_sol_mat[W, 3] = cell_W;
        pressure_sol_mat[P, 3] = cell_P;
        pressure_sol_mat[E, 3] = cell_E;
        pressure_sol_mat[N, 3] = cell_N;
        
        A_cell_S = cell_S - N_y;
        A_cell_W = cell_W - N_y;
        A_cell_P = cell_P - N_y;
        A_cell_E = cell_E - N_y;
        A_cell_N = cell_N - N_y;
        
        pressure_sol_mat[S, 1] = A_cell_S;
        pressure_sol_mat[W, 1] = A_cell_W;
        pressure_sol_mat[P, 1] = A_cell_P;
        pressure_sol_mat[E, 1] = A_cell_E;
        pressure_sol_mat[N, 1] = A_cell_N;

#define fine structure of column 1 (cut the linkage of boundary cells)
for a_x in range(1, N_x -1):
    for a_y in range(0, N_y):
        cell_P = int(a_x*N_y + a_y);
        
        location_sol_mat = (cell_P-N_y)*5;
        S = location_sol_mat;
        W = S + 1;
        P = S + 2;
        E = S + 3;
        N = S + 4;
        
        if cell_P == N_y: #south-west corner
            pressure_sol_mat[S, 1] = 0;
            pressure_sol_mat[W, 1] = 0;
            pressure_sol_mat[S, 3] = 0;
            pressure_sol_mat[W, 3] = 0;
        elif cell_P == 2*N_y - 1: #north-west corner
            pressure_sol_mat[W, 1] = 0;
            pressure_sol_mat[N, 1] = 0;
            pressure_sol_mat[W, 3] = 0;
            pressure_sol_mat[N, 3] = 0;
        elif cell_P == (N_x-2)*N_y: #south-east corner
            pressure_sol_mat[S, 1] = 0;
            pressure_sol_mat[E, 1] = 0;
            pressure_sol_mat[S, 3] = 0;
            pressure_sol_mat[E, 3] = 0;
        elif cell_P == (N_x-1)*N_y -1: #north-east corner
            pressure_sol_mat[E, 1] = 0;
            pressure_sol_mat[N, 1] = 0;
            pressure_sol_mat[E, 3] = 0;
            pressure_sol_mat[N, 3] = 0;
        if cell_P in inlet_pressure_cells:
            pressure_sol_mat[W, 1] = 0;
            pressure_sol_mat[W, 3] = 0;
        elif cell_P in outlet_pressure_cells:
            pressure_sol_mat[E, 1] = 0;
            pressure_sol_mat[E, 3] = 0;
        elif cell_P in bottom_pressure_wall:
            pressure_sol_mat[S, 1] = 0;
            pressure_sol_mat[S, 3] = 0;
        elif cell_P in top_pressure_wall:
            pressure_sol_mat[N, 1] = 0;
            pressure_sol_mat[N, 3] = 0;

In [ ]:
#define U-VELOCIY solution matrix in a similar fashion

u_vel_sol_mat = np.zeros(((N_x-1)*N_y*5, 4));

#define column 0
for a_x in range(1, N_x):
    for a_y in range(0, N_y):
        cell_P = int(a_x*N_y + a_y);
        index_location_1 = (cell_P-N_y)*5 ;
        index_location_2 = index_location_1 + 5; #for 2D
        u_vel_sol_mat[index_location_1 : index_location_2, 0] = cell_P;

#define boundary cells list
#corners, inlet, outlet and top, bottom walls

u_vel_corners = [N_y, 2*N_y - 1, (N_x-1)*N_y, N_x*N_y -1 ];
inlet_u_vel_cells = np.arange(N_y + 1, 2*N_y - 1, 1);
outlet_u_vel_cells = np.arange((N_x-1)*N_y + 1, N_x*N_y -1, 1);
bottom_u_vel_wall = np.arange(2*N_y, (N_x-1)*N_y, N_y);
top_u_vel_wall = np.arange(3*N_y -1, N_x*N_y - 1, N_y);

print(u_vel_corners);
print(inlet_u_vel_cells);
print(outlet_u_vel_cells);
print(bottom_u_vel_wall);
print(top_u_vel_wall);

#define gross structure of column 1
for a_x in range(1, N_x):
    for a_y in range(0, N_y):
        cell_P = int(a_x*N_y + a_y);
        
        location_sol_mat = (cell_P-N_y)*5;
        S = location_sol_mat;
        W = S + 1;
        P = S + 2;
        E = S + 3;
        N = S + 4;
        
        cell_S = cell_P -1;
        cell_W = cell_P - N_y;
        cell_E = cell_P + N_y;
        cell_N = cell_P + 1;
        
        u_vel_sol_mat[S, 3] = cell_S;
        u_vel_sol_mat[W, 3] = cell_W;
        u_vel_sol_mat[P, 3] = cell_P;
        u_vel_sol_mat[E, 3] = cell_E;
        u_vel_sol_mat[N, 3] = cell_N;
        
        A_cell_S = cell_S - N_y;
        A_cell_W = cell_W - N_y;
        A_cell_P = cell_P - N_y;
        A_cell_E = cell_E - N_y;
        A_cell_N = cell_N - N_y;
        
        u_vel_sol_mat[S, 1] = A_cell_S;
        u_vel_sol_mat[W, 1] = A_cell_W;
        u_vel_sol_mat[P, 1] = A_cell_P;
        u_vel_sol_mat[E, 1] = A_cell_E;
        u_vel_sol_mat[N, 1] = A_cell_N;
        
#define fine structure of column 1 (cut the linkage of boundary cells)
for a_x in range(1, N_x):
    for a_y in range(0, N_y):
        cell_P = int(a_x*N_y + a_y);
        
        location_sol_mat = (cell_P-N_y)*5;
        S = location_sol_mat;
        W = S + 1;
        P = S + 2;
        E = S + 3;
        N = S + 4;
        
        cell_S = cell_P -1;
        cell_W = cell_P - N_y;
        cell_E = cell_P + N_y;
        cell_N = cell_P + 1;
        
        if cell_P == N_y: #south-west corner
            u_vel_sol_mat[S, 1] = 0;
            u_vel_sol_mat[W, 1] = 0;
            u_vel_sol_mat[S, 3] = 0;
            u_vel_sol_mat[W, 3] = 0;
        elif cell_P == 2*N_y - 1: #north-west corner
            u_vel_sol_mat[W, 1] = 0;
            u_vel_sol_mat[N, 1] = 0;
            u_vel_sol_mat[W, 3] = 0;
            u_vel_sol_mat[N, 3] = 0;
        elif cell_P == (N_x-1)*N_y: #south-east corner
            u_vel_sol_mat[S, 1] = 0;
            u_vel_sol_mat[E, 1] = 0;
            u_vel_sol_mat[S, 3] = 0;
            u_vel_sol_mat[E, 3] = 0;
        elif cell_P == N_x*N_y -1: #north-east corner
            u_vel_sol_mat[E, 1] = 0;
            u_vel_sol_mat[N, 1] = 0;
            u_vel_sol_mat[E, 3] = 0;
            u_vel_sol_mat[N, 3] = 0;
        if cell_P in inlet_u_vel_cells:
            u_vel_sol_mat[W, 1] = 0;
            u_vel_sol_mat[W, 3] = 0;
        elif cell_P in outlet_u_vel_cells:
            u_vel_sol_mat[E, 1] = 0;
            u_vel_sol_mat[E, 3] = 0;
        elif cell_P in bottom_u_vel_wall:
            u_vel_sol_mat[S, 1] = 0;
            u_vel_sol_mat[S, 3] = 0;
        elif cell_P in top_u_vel_wall:
            u_vel_sol_mat[N, 1] = 0;
            u_vel_sol_mat[N, 3] = 0;

In [ ]:
#define V-VELOCIY solution matrix in a similar fashion

v_vel_sol_mat =np.zeros(((N_y-1)*(N_x-2)*5, 4));

#define column 0
cell_location = 0; #move through the entire matrix
for a_x in range(1, N_x-1):
    for a_y in range(1, N_y):
        cell_P = int(a_x*(N_y+1) + a_y);
        index_location_1 = cell_location*5;
        index_location_2 = index_location_1 + 5;
        v_vel_sol_mat[index_location_1 : index_location_2, 0] = cell_P;
        
        cell_location = cell_location + 1;

#define boundary cells list
#corners, inlet, outlet and top, bottom walls

v_vel_corners = [N_y + 2, 2*N_y, (N_x-2)*(N_y+1) + 1, (N_x-1)*(N_y+1) -2];
inlet_v_vel_cells = np.arange(N_y + 3, 2*N_y, 1);
outlet_v_vel_cells = np.arange((N_x-2)*(N_y+1) + 2, (N_x-1)*(N_y+1) -2, 1);
bottom_v_vel_wall = np.arange(2*N_y + 3, (N_x-2)*(N_y+1) + 1, N_y + 1);
top_v_vel_wall = np.arange(3*N_y + 1,(N_x-1)*(N_y+1) -2, N_y + 1);

print(v_vel_corners);
print(inlet_v_vel_cells);
print(outlet_v_vel_cells);
print(bottom_v_vel_wall);
print(top_v_vel_wall);

#define gross structure of column 1
cell_location = 0;
for a_x in range(1, N_x-1):
    for a_y in range(1, N_y):
        cell_P = int(a_x*(N_y+1) + a_y);
        
        location_sol_mat = cell_location*5;
        S = location_sol_mat;
        W = S + 1;
        P = S + 2;
        E = S + 3;
        N = S + 4;
        
        cell_S = cell_P -1;
        cell_W = cell_P - (N_y+1);
        cell_E = cell_P + (N_y+1);
        cell_N = cell_P + 1;
        
        v_vel_sol_mat[S, 3] = cell_S;
        v_vel_sol_mat[W, 3] = cell_W;
        v_vel_sol_mat[P, 3] = cell_P;
        v_vel_sol_mat[E, 3] = cell_E;
        v_vel_sol_mat[N, 3] = cell_N;
        
        A_cell_P = int((a_x-1)*(N_y-1) + (a_y-1));
        A_cell_S = int(A_cell_P -1);
        A_cell_W = int(A_cell_P - (N_y-1))
        A_cell_E = int(A_cell_P + (N_y-1))
        A_cell_N = int(A_cell_P +1);
        
        v_vel_sol_mat[S, 1] = A_cell_S;
        v_vel_sol_mat[W, 1] = A_cell_W;
        v_vel_sol_mat[P, 1] = A_cell_P;
        v_vel_sol_mat[E, 1] = A_cell_E;
        v_vel_sol_mat[N, 1] = A_cell_N;
        
        cell_location = cell_location + 1;
        
#define fine structure of column 1 (cut the linkage of boundary cells)
cell_location = 0;
for a_x in range(1, N_x-1):
    for a_y in range(1, N_y):
        cell_P = int(a_x*(N_y+1) + a_y);
        
        location_sol_mat = cell_location*5;
        S = location_sol_mat;
        W = S + 1;
        P = S + 2;
        E = S + 3;
        N = S + 4;
        
        cell_S = cell_P -1;
        cell_W = cell_P - (N_y+1);
        cell_E = cell_P + (N_y+1);
        cell_N = cell_P + 1;
        
        if cell_P == N_y + 2: #south-west corner
            v_vel_sol_mat[S, 1] = 0;
            v_vel_sol_mat[W, 1] = 0;
            v_vel_sol_mat[S, 3] = 0;
            v_vel_sol_mat[W, 3] = 0;
        elif cell_P == 2*N_y: #north-west corner
            v_vel_sol_mat[W, 1] = 0;
            v_vel_sol_mat[N, 1] = 0;
            v_vel_sol_mat[W, 3] = 0;
            v_vel_sol_mat[N, 3] = 0;
        elif cell_P == (N_x-2)*(N_y+1) + 1: #south-east corner
            v_vel_sol_mat[S, 1] = 0;
            v_vel_sol_mat[E, 1] = 0;
            v_vel_sol_mat[S, 3] = 0;
            v_vel_sol_mat[E, 3] = 0;
        elif cell_P == (N_x-1)*(N_y+1) -2: #north-east corner
            v_vel_sol_mat[E, 1] = 0;
            v_vel_sol_mat[N, 1] = 0;
            v_vel_sol_mat[E, 3] = 0;
            v_vel_sol_mat[N, 3] = 0;
        if cell_P in inlet_v_vel_cells:
            v_vel_sol_mat[W, 1] = 0;
            v_vel_sol_mat[W, 3] = 0;
        elif cell_P in outlet_v_vel_cells:
            v_vel_sol_mat[E, 1] = 0;
            v_vel_sol_mat[E, 3] = 0;
        elif cell_P in bottom_v_vel_wall:
            v_vel_sol_mat[S, 1] = 0;
            v_vel_sol_mat[S, 3] = 0;
        elif cell_P in top_v_vel_wall:
            v_vel_sol_mat[N, 1] = 0;
            v_vel_sol_mat[N, 3] = 0;
            
        cell_location = cell_location + 1;

In [ ]:
#define physical data of the fluid (water)

rho = 1; #g/ cm^3 density
eta = 1.002; #g/cm*s dynamic viscosity
D = eta/del_l;

#define the source terms
pressure_source = np.zeros((1, (N_x - 2)*N_y));
u_vel_source = np.zeros((1, (N_x-1)*N_y));
v_vel_source = np.zeros((1, (N_y-1)*(N_x-2)));

In [ ]:
#must construct the UD scheme for U-VELOCITY and V-VELOCITY EQUATIONS
#this will be used inside the iterative SIMPLE algorithm and will require continuous update
#after each iteration

#the algorithm is made so that as the matrix is created also the source term is created

In [ ]:
#U-VELOCITY MATRIX - define the structure of the UD scheme implementation
#implemented as a function for compactness of the SIMPLE algorithm

#WE INCLUDE THE U-VELOCITY BOUNDARY CONDITION

def update_u_vel_sol_mat(u_vel_sol_mat, u_vel_source, u_vel, v_vel, pressure, N_x, N_y, rho, 
                         D, u_vel_corners, inlet_u_vel_cells, outlet_u_vel_cells, 
                         bottom_u_vel_wall, top_u_vel_wall):
    
    cell_location = 0; #this variable will also give the location in the source term
    for a_x in range(1, N_x):
        for a_y in range(0, N_y):
            cell_P = int(a_x*N_y + a_y);
        
            location_sol_mat = cell_location*5;
            S = location_sol_mat;
            W = S + 1;
            P = S + 2;
            E = S + 3;
            N = S + 4;
        
            #location of the v-velocities needed in flux calculation
            v_S_1 = int(u_vel_sol_mat[W, 3] + a_x);
            v_S_2 = int(u_vel_sol_mat[P, 3] + a_x);
            v_N_1 = int(u_vel_sol_mat[W, 3] + a_x + 1);
            v_N_2 = int(u_vel_sol_mat[P, 3] + a_x + 1);
        
            #location of pressure cells
            p_W = int(cell_P - N_y);
            p_E = cell_P;
        
            #calculation of all the cell fluxes 
            F_e = rho*(u_vel[0, int(u_vel_sol_mat[P, 3])] + u_vel[0, int(u_vel_sol_mat[E, 3])])/2;
            F_w = rho*(u_vel[0, int(u_vel_sol_mat[P, 3])] + u_vel[0, int(u_vel_sol_mat[W, 3])])/2;
            F_n = rho*(v_vel[0, v_N_1] + v_vel[0, v_N_2])/2;
            F_s = rho*(v_vel[0, v_S_1] + v_vel[0, v_S_2])/2;
        
            #first we deal with the corners
            if cell_P == u_vel_corners[0]: #SW corner
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since u_wall = 0
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_n + F_e - F_w + 3*D
                    a_E = - D;
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = -pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = 0; #since u_wall = 0
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_e - F_w - F_s + 3*D;
                    a_E = - D;
                    a_N = F_n - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since u_wall = 0
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_n - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = 0; #since u_wall = 0
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = - F_s - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = F_n - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
        
            elif cell_P == u_vel_corners[1]: #NW corner
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D;
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_n + F_e - F_w  + 3*D
                    a_E = - D;
                    a_N = 0; #since u_wall = 0
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = - D; 
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_e - F_w - F_s + 3*D
                    a_E = - D;
                    a_N = 0; #since u_wall = 0
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_n - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = 0; #since u_wall = 0
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = - F_s - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = 0; #since u_wall = 0
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
        
            elif cell_P == u_vel_corners[2]: #SE corner
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since u_wall = 0
                    a_W = -F_w -D; 
                    a_P = F_n + F_e + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] =- pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = 0; #since u_wall = 0
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = F_n - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since u_wall = 0
                    a_W = -D; 
                    a_P = F_n + F_e - F_w  + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = 0; #since u_wall = 0
                    a_W = -D; 
                    a_P = F_e - F_s - F_w  + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = F_n - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
        
            elif cell_P == u_vel_corners[3]: #NE corner
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D;
                    a_W = -F_w -D;
                    a_P = F_n + F_e  + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = 0; #since u_wall = 0
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = - D; 
                    a_W = -F_w -D;
                    a_P = F_e - F_s + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = 0; #since u_wall = 0
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = -D;
                    a_P = F_n + F_e - F_w  + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = 0; #since u_wall = 0
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = -D;
                    a_P = F_e - F_s - F_w  + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = 0; #since u_wall = 0
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                
            #then we deal with inlet, outlet, and walls
            elif cell_P in inlet_u_vel_cells:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_n + F_e - F_w + 3*D;
                    a_E = - D;
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = -D;
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_e - F_w - F_s + 3*D;
                    a_E = - D;
                    a_N = F_n - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s -D;
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = F_n - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D;
                    a_W = 0; #since u_W = u_P under the zero gradient condition
                    a_P = - F_s - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = F_n - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
        
            elif cell_P in outlet_u_vel_cells:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s -D;
                    a_W = -F_w -D; 
                    a_P = F_n + F_e + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = F_n - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s -D;
                    a_W = -D; 
                    a_P = F_n + F_e - F_w  + 3*D
                    a_E = 0; #since u_E = u_P under the zero gradient condition
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D;
                    a_W = -D; 
                    a_P = F_e - F_s - F_w  + 3*D
                    a_E = 0; #since u_W = u_P under the zero gradient condition
                    a_N = F_n - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;
                
                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
            
            elif cell_P in bottom_u_vel_wall:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since u_wall = 0
                    a_W = -F_w -D;
                    a_P = F_n + F_e + 4*D
                    a_E = - D;
                    a_N = - D;
                
                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = 0; #since u_wall = 0
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 4*D
                    a_E = - D;
                    a_N = F_n - D;

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since u_wall = 0
                    a_W = -D; 
                    a_P = F_n - F_w  + 4*D
                    a_E = F_e - D;
                    a_N = - D;

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = 0; #since u_wall = 0
                    a_W = -D;
                    a_P = - F_s - F_w  + 4*D
                    a_E = F_e - D;
                    a_N = F_n - D;

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];

            elif cell_P in top_u_vel_wall:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D;
                    a_W = -F_w -D;
                    a_P = F_n + F_e  + 4*D
                    a_E = - D;
                    a_N = 0; #since u_wall = 0

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = - D; 
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 4*D
                    a_E = - D;
                    a_N = 0; #since u_wall = 0

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = -D; 
                    a_P = F_n - F_w  + 4*D
                    a_E = F_e - D;
                    a_N = 0; #since u_wall = 0

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = -D; 
                    a_P = - F_s - F_w  + 4*D
                    a_E = F_e - D;
                    a_N = 0; #since u_wall = 0

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];

            #interior
            else:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D;
                    a_W = -F_w -D;
                    a_P = F_n + F_e  + 4*D;
                    a_E = - D;
                    a_N = -D;

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = - D; 
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 4*D;
                    a_E = - D;
                    a_N = F_n -D;

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = -D; 
                    a_P = F_n - F_w  + 4*D;
                    a_E = F_e - D;
                    a_N = -D;

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = -D; 
                    a_P = - F_s - F_w  + 4*D;
                    a_E = F_e - D;
                    a_N = F_n -D;

                    u_vel_sol_mat[S, 2] = a_S;
                    u_vel_sol_mat[W, 2] = a_W;
                    u_vel_sol_mat[P, 2] = a_P;
                    u_vel_sol_mat[E, 2] = a_E;
                    u_vel_sol_mat[N, 2] = a_N;

                    u_vel_source[0, cell_location] = - pressure[0, p_E] + pressure[0, p_W];

            cell_location = cell_location + 1;
            
    return u_vel_sol_mat, u_vel_source

In [ ]:
#V-VELOCITY MATRIX - define the structure of the UD scheme implementation
#implemented as a function for compactness of the SIMPLE algorithm

#WE INCLUDE THE V-VELOCITY BOUNDARY CONDITIONS

def update_v_vel_sol_mat(v_vel_sol_mat, v_vel_source, u_vel, v_vel, pressure, N_x, N_y, rho, 
                         D, v_vel_corners, inlet_v_vel_cells, outlet_v_vel_cells, 
                         bottom_v_vel_wall, top_v_vel_wall):

    cell_location = 0; #this variable will also give the location in the source term
    for a_x in range(1, N_x-1):
        for a_y in range(1, N_y):
            cell_P = int(a_x*(N_y+1) + a_y);

            location_sol_mat = cell_location*5;
            S = location_sol_mat;
            W = S + 1;
            P = S + 2;
            E = S + 3;
            N = S + 4;

            #location of the u-velocities needed in flux calculation
            u_W_1 = int(v_vel_sol_mat[P, 3] - a_x);
            u_W_2 = int(v_vel_sol_mat[P, 3] - a_x -1);
            u_E_1 = int(v_vel_sol_mat[E, 3] - a_x - 1);
            u_E_2 = int(v_vel_sol_mat[E, 3] - a_x - 2);

            #location of pressure cells
            p_N = int(cell_P - a_x);
            p_S = int(cell_P - a_x - 1);

            #calculation of all the cell fluxes 
            F_n = rho*(v_vel[0, int(v_vel_sol_mat[N, 3])] + v_vel[0, int(v_vel_sol_mat[P, 3])])/2;
            F_s = rho*(v_vel[0, int(v_vel_sol_mat[P, 3])] + v_vel[0, int(v_vel_sol_mat[S, 3])])/2;
            F_e = rho*(u_vel[0, u_E_1] + u_vel[0, u_E_2])/2;
            F_w = rho*(u_vel[0, u_W_1] + u_vel[0, u_W_2])/2;

            #first we deal with the corners
            if cell_P == v_vel_corners[0]: #SW corner
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since v_wall = 0
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_n + F_e - F_w + 3*D
                    a_E = - D;
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = 0; #since v_wall = 0
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_e - F_w - F_s + 3*D
                    a_E = - D;
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since v_wall = 0
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_n - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = 0; #since v_wall = 0
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = - F_s - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            elif cell_P == v_vel_corners[1]: #NW corner
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D;
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_n + F_e - F_w  + 3*D
                    a_E = - D;
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = - D; 
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_e - F_w - F_s + 3*D
                    a_E = - D;
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_n - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = - F_s - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            elif cell_P == v_vel_corners[2]: #SE corner
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since v_wall = 0
                    a_W = -F_w -D; 
                    a_P = F_n + F_e + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = 0; #since v_wall = 0
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since v_wall = 0
                    a_W = -D; 
                    a_P = F_n + F_e - F_w  + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = 0; #since v_wall = 0
                    a_W = -D; 
                    a_P = F_e - F_s - F_w  + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            elif cell_P == v_vel_corners[3]: #NE corner
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D;
                    a_W = -F_w -D;
                    a_P = F_n + F_e  + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = - D; 
                    a_W = -F_w -D;
                    a_P = F_e - F_s + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = -D;
                    a_P = F_n + F_e - F_w  + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = -D;
                    a_P = F_e - F_s - F_w  + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            #then we deal with inlet, outlet, and walls
            elif cell_P in inlet_v_vel_cells:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_n + F_e - F_w + 3*D;
                    a_E = - D;
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = -D;
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_e - F_w - F_s + 3*D;
                    a_E = - D;
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s -D;
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = F_n - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D;
                    a_W = 0; #since v_W = v_P under the zero gradient condition
                    a_P = - F_s - F_w  + 3*D
                    a_E = F_e - D;
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            elif cell_P in outlet_v_vel_cells:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s -D;
                    a_W = -F_w -D; 
                    a_P = F_n + F_e + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s -D;
                    a_W = -D; 
                    a_P = F_n + F_e - F_w  + 3*D
                    a_E = 0; #since v_E = v_P under the zero gradient condition
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D;
                    a_W = -D; 
                    a_P = F_e - F_s - F_w  + 3*D
                    a_E = 0; #since v_W = v_P under the zero gradient condition
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            elif cell_P in bottom_v_vel_wall:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since v_wall = 0
                    a_W = -F_w -D;
                    a_P = F_n + F_e + 4*D
                    a_E = - D;
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = 0; #since v_wall = 0
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 4*D
                    a_E = - D;
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = 0; #since v_wall = 0
                    a_W = -D; 
                    a_P = F_n - F_w  + 4*D
                    a_E = F_e - D;
                    a_N = - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = 0; #since v_wall = 0
                    a_W = -D;
                    a_P = - F_s - F_w  + 4*D
                    a_E = F_e - D;
                    a_N = F_n - D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            elif cell_P in top_v_vel_wall:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D;
                    a_W = -F_w -D;
                    a_P = F_n + F_e  + 4*D
                    a_E = - D;
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = - D; 
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 4*D
                    a_E = - D;
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = -D; 
                    a_P = F_n - F_w  + 4*D
                    a_E = F_e - D;
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = -D; 
                    a_P = - F_s - F_w  + 4*D
                    a_E = F_e - D;
                    a_N = 0; #since v_wall = 0

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            #interior
            else:
                if (F_e + F_w) >= 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D;
                    a_W = -F_w -D;
                    a_P = F_n + F_e  + 4*D;
                    a_E = - D;
                    a_N = -D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) >= 0 and (F_n + F_s) < 0:
                    a_S = - D; 
                    a_W = -F_w -D; 
                    a_P = F_e - F_s + 4*D;
                    a_E = - D;
                    a_N = F_n -D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) >= 0:
                    a_S = -F_s - D; 
                    a_W = -D; 
                    a_P = F_n - F_w  + 4*D;
                    a_E = F_e - D;
                    a_N = -D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];
                elif (F_e + F_w) < 0 and (F_n + F_s) < 0:
                    a_S = -D; 
                    a_W = -D; 
                    a_P = - F_s - F_w  + 4*D;
                    a_E = F_e - D;
                    a_N = F_n -D;

                    v_vel_sol_mat[S, 2] = a_S;
                    v_vel_sol_mat[W, 2] = a_W;
                    v_vel_sol_mat[P, 2] = a_P;
                    v_vel_sol_mat[E, 2] = a_E;
                    v_vel_sol_mat[N, 2] = a_N;

                    v_vel_source[0, cell_location] = - pressure[0, p_N] + pressure[0, p_S];

            cell_location = cell_location + 1;
    
    return v_vel_sol_mat, v_vel_source

In [ ]:
#PRESSURE CORRECTION MATRIX
#In this part of the code we don't adopt the same convention used in the original creation of
#the pressure solution matrix, we adopt a convention based on the pressure_sol_mat structure
#also needed to simplify the connection to the u_vel_sol_mat and the v_vel_sol_mat

#WE INCLUDE THE PRESSURE BOUNDARY CONDITION
def update_pressure_sol_mat(pressure_sol_mat, pressure_source, N_x, N_y, u_vel_sol_mat,
                           v_vel_sol_mat, u_vel, v_vel, pressure_corners, inlet_pressure_cells,
                           outlet_pressure_cells, bottom_pressure_wall, top_pressure_wall):
    cell_location = 0;
    for a_x in range(0, N_x - 2):
        for a_y in range(0, N_y):
            cell_P = int((a_x + 1)*N_y + a_y);     #needed for BC statements 
            cell_P_new = int(a_x*N_y + a_y);       #for pressure_sol_mat work

            location_sol_mat = cell_location*5;
            S = location_sol_mat;
            W = S + 1;
            P = S + 2;
            E = S + 3;
            N = S + 4;

            #define the location of the velocity central coefficients for the interior cells
            #we deal with logical statements with the boundaries in order to inplement BC
            loc_u_W = int(cell_P_new*5 + 2);
            loc_u_E = int((cell_P_new + N_y)*5 + 2);
            loc_v_N = int((cell_P_new - a_x)*5 + 2);
            loc_v_S = int((cell_P_new - a_x - 1)*5 + 2);

            #define the location of the velocities themeselves
            #needed in the continuity imbalance source term
            u_W = cell_P;
            u_E = int(cell_P + N_y);
            v_S = int(cell_P + a_x + 1);
            v_N = int(cell_P + a_x + 2);

            if cell_P == pressure_corners[0]: #SW corner
                a_S = 0 #p' = 0 since velocity is fixed, no pressure correction
                a_W = 0 #p' = 0 since pressure is set
                a_E = -(1/u_vel_sol_mat[loc_u_E, 2]);
                a_N = -(1/v_vel_sol_mat[loc_v_N, 2]);
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);
            elif cell_P == pressure_corners[1]: #NW corner
                a_S = -(1/v_vel_sol_mat[loc_v_S, 2]);
                a_W = 0 #p' = 0 since pressure is set
                a_E = -(1/u_vel_sol_mat[loc_u_E, 2]);
                a_N = 0; #p' = 0 since velocity is fixed, no pressure correction
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);
            elif cell_P == pressure_corners[2]: #SE corner
                a_S = 0; #p' = 0 since velocity is fixed, no pressure correction
                a_W = -(1/u_vel_sol_mat[loc_u_W, 2]);
                a_E = 0; #p' = 0 since pressure is set
                a_N = -(1/v_vel_sol_mat[loc_v_N, 2]); 
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);
            elif cell_P == pressure_corners[3]: #NE corner
                a_S = -(1/v_vel_sol_mat[loc_v_S, 2]);
                a_W = -(1/u_vel_sol_mat[loc_u_W, 2]);
                a_E = 0; #p' = 0 since pressure is set
                a_N = 0; #p' = 0 since velocity is fixed, no pressure correction
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);
            #inlet, outlet, walls
            elif cell_P in inlet_pressure_cells:
                a_S = -(1/v_vel_sol_mat[loc_v_S, 2]);
                a_W = 0; #p' = 0 since pressure is set
                a_E = -(1/u_vel_sol_mat[loc_u_E, 2]);
                a_N = -(1/v_vel_sol_mat[loc_v_N, 2]);
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);
            elif cell_P in outlet_pressure_cells:
                a_S = -(1/v_vel_sol_mat[loc_v_S, 2]);
                a_W = -(1/u_vel_sol_mat[loc_u_W, 2]); 
                a_E = 0; #p' = 0 since pressure is set
                a_N = -(1/v_vel_sol_mat[loc_v_N, 2]);
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);
            elif cell_P in bottom_pressure_wall:
                a_S = 0; #p' = 0 since velocity is fixed, no pressure correction
                a_W = -(1/u_vel_sol_mat[loc_u_W, 2]); 
                a_E = -(1/u_vel_sol_mat[loc_u_E, 2]); 
                a_N = -(1/v_vel_sol_mat[loc_v_N, 2]);
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);
            elif cell_P in top_pressure_wall:
                a_S = -(1/v_vel_sol_mat[loc_v_S, 2]);
                a_W = -(1/u_vel_sol_mat[loc_u_W, 2]); 
                a_E = -(1/u_vel_sol_mat[loc_u_E, 2]); 
                a_N = 0; #p' = 0 since velocity is fixed, no pressure correction
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);
            #interior
            else:
                a_S = -(1/v_vel_sol_mat[loc_v_S, 2]);
                a_W = -(1/u_vel_sol_mat[loc_u_W, 2]); 
                a_E = -(1/u_vel_sol_mat[loc_u_E, 2]); 
                a_N = -(1/v_vel_sol_mat[loc_v_N, 2]);
                a_P = -(a_S + a_N + a_E + a_W);

                pressure_sol_mat[S, 2] = a_S;
                pressure_sol_mat[W, 2] = a_W;
                pressure_sol_mat[P, 2] = a_P;
                pressure_sol_mat[E, 2] = a_E;
                pressure_sol_mat[N, 2] = a_N;

                pressure_source[0, cell_location] = -(u_vel[0, u_E] - u_vel[0, u_W] + v_vel[0, v_N] - v_vel[0, v_S]);

            cell_location = cell_location + 1;
            
    return pressure_sol_mat, pressure_source

In [ ]:
u_vel_sol_mat, u_vel_source = update_u_vel_sol_mat(u_vel_sol_mat, u_vel_source, u_vel, v_vel, pressure, N_x, N_y, rho, 
                         D, u_vel_corners, inlet_u_vel_cells, outlet_u_vel_cells, 
                         bottom_u_vel_wall, top_u_vel_wall);
    
v_vel_sol_mat, v_vel_source = update_v_vel_sol_mat(v_vel_sol_mat, v_vel_source, u_vel, v_vel, pressure, N_x, N_y, rho, 
                     D, v_vel_corners, inlet_v_vel_cells, outlet_v_vel_cells, 
                     bottom_v_vel_wall, top_v_vel_wall);

pressure_sol_mat, pressure_source = update_pressure_sol_mat(pressure_sol_mat, pressure_source, N_x, N_y, u_vel_sol_mat,
                       v_vel_sol_mat, u_vel, v_vel, pressure_corners, inlet_pressure_cells,
                       outlet_pressure_cells, bottom_pressure_wall, top_pressure_wall);

df1 = pd.DataFrame(pressure_sol_mat);
df1.to_csv("pressure_sol_mat.csv")

df2 = pd.DataFrame(u_vel_sol_mat);
df2.to_csv("u_vel_sol_mat.csv")

df3 = pd.DataFrame(v_vel_sol_mat);
df3.to_csv("v_vel_sol_mat.csv")

df4 = pd.DataFrame(pressure_source);
df4.to_csv("pressure_source.csv")

df5 = pd.DataFrame(u_vel_source);
df5.to_csv("u_vel_source.csv")

df6 = pd.DataFrame(v_vel_source);
df6.to_csv("v_vel_source.csv")

In [ ]:
#CGS solver

#must be modified since the domain size and the number of active cells are not the same 
def CGS_solver(sol_mat, b_source, guess_0):
    a, b = np.shape(b_source); #size of the system, needed for stopping the algorithm
    columns = int(b);
    
    def matrix_product_5bands(matrix, vector): #specifically for the sparse sol_mat matrix
        rows, columns = np.shape(vector);
        product = np.zeros((1, int(columns))); #list of zeros
        
        for a in range(0, columns):
            t_1 = int(a*5);
            t_2 = int(t_1 +1);
            t_3 = int(t_2 +1);
            t_4 = int(t_3 +1);
            t_5 = int(t_4 +1);
            
            S = int(sol_mat[t_1, 1]);
            W = int(sol_mat[t_2, 1]);
            P = int(sol_mat[t_3, 1]);
            E = int(sol_mat[t_4, 1]);
            N = int(sol_mat[t_5, 1]);
            
            term = sol_mat[t_1, 2]*vector[0, S] + sol_mat[t_2, 2]*vector[0, W] + sol_mat[t_3, 2]*vector[0, P] + sol_mat[t_4, 2]*vector[0, E] + sol_mat[t_5, 2]*vector[0, N];
            
            product[0, a] = term
            
        return product
    
    #initialize with 0th residual
    residual_0 = b_source - matrix_product_5bands(sol_mat, guess_0);
    
    #initialize direction vectors
    dir_vec = residual_0;
    C_dir_vec = residual_0;
    
    #initialize L-2 global residual with residual 0
    trans_residual_0 = residual_0.T;
    R2 = np.sqrt(residual_0.dot(trans_residual_0))
    
    #initialize residual for iteration
    residual = residual_0
    
    #initialize solution
    solution = guess_0
    #this will be solution at 0th iteration
    
    #set tolerance
    tolerance = 10**-5 #ADJUSTED ACCORDING OT THE PROBLEM
    #reference to control the convergence
    ref = 105;
    #number of iterations
    iter_N = 0;
    
    while R2 > tolerance:
        trans_residual = residual.T;
        
        #alpha^n+1, coefficient needed in solution update
        alpha = (residual_0.dot(trans_residual))/(residual_0.dot(matrix_product_5bands(sol_mat, dir_vec).T));
        
        #vector needed to update solution and account for a-symmetric nature of the solution, G^n+1
        G_vec = C_dir_vec - alpha*(matrix_product_5bands(sol_mat, dir_vec));
        
        #update the solution
        solution = solution + alpha*(C_dir_vec + G_vec);
        
        #new residual and new global residual
        residual_n_1 = b_source - matrix_product_5bands(sol_mat, solution);
        trans_residual_n_1 = residual_n_1.T
        R2_temporary = np.sqrt(residual_n_1.dot(residual_n_1.T));
        R2 = R2_temporary[0, 0];
        
        #beta^n+1, needed in conjugate direction, search direction vector update
        beta = (residual_0.dot(trans_residual_n_1))/(residual_0.dot(trans_residual));
        
        #update conjugate direction
        C_dir_vec = residual_n_1 + beta*G_vec;
        
        #update search direction
        dir_vec = C_dir_vec + beta*(G_vec + beta*dir_vec);
        
        residual = residual_n_1
        
        print(solution[0, ref], R2, iter_N)
        
        iter_N = iter_N + 1
        
    return solution

In [ ]:
#iteration of SIMPLE

#initialize the solution vectors
pressure_correction =  np.zeros((1, (N_x - 2)*N_y));
solution_u = np.zeros((1, (N_x-1)*N_y));
solution_v = np.zeros((1, (N_y-1)*(N_x-2)));

#define global residual and tolerance
R_G_u = 5;
R_G_v = 5;
tolerance = 10**(-3);
#required in global residual computation
residual_u = 0;
residual_v = 0;

#define under-relaxation factor
under_relax = 0.3;

#define pressure_correction trans
pressure_correction_trans = np.zeros((1, N_x*N_y)); #needed for the correction of u-vel and v-vel
                                                    #this vector will contain all the corrections (including inlet and outlet)
                                                    #at each iteration
        
#locations of boundary cells, physical parameters, u, v, and p matrices have already been defined

while (R_G_u > tolerance) or (R_G_v > tolerance):
    
    #results form previous iteration
    #needed for under-relaxation
    u_vel_n = u_vel;
    v_vel_n = v_vel;
    pressure_n = pressure;
    
    #create the solution matrices for momentum
    u_vel_sol_mat, u_vel_source = update_u_vel_sol_mat(u_vel_sol_mat, u_vel_source, u_vel, v_vel, pressure, N_x, N_y, rho, 
                         D, u_vel_corners, inlet_u_vel_cells, outlet_u_vel_cells, 
                         bottom_u_vel_wall, top_u_vel_wall);
    
    v_vel_sol_mat, v_vel_source = update_v_vel_sol_mat(v_vel_sol_mat, v_vel_source, u_vel, v_vel, pressure, N_x, N_y, rho, 
                         D, v_vel_corners, inlet_v_vel_cells, outlet_v_vel_cells, 
                         bottom_v_vel_wall, top_v_vel_wall);
    
    #solve for the velocity
    solution_u = CGS_solver(u_vel_sol_mat, u_vel_source, solution_u);
    print("solution_u: ", solution_u)
    solution_v = CGS_solver(v_vel_sol_mat, v_vel_source, solution_v);
    print("solution_v: ", solution_v)
    
    #FIRST ASSIGMENT BEFORE CORRECTION
    
    #u-velocity corrected velocities + application of zero gradient boundary
    #apply the correction only to the cells strictly in the interior
    #recall from the u_sol_mat creation
    u_cell_location = 0; #this will provide the location of the cell in the solution vector!
    for a_x in range(1, N_x):
        for a_y in range(0, N_y):
            cell_P = int(a_x*N_y + a_y);
            
            #u-vel correction
            u_vel[0, cell_P] = solution_u[0, u_cell_location];
            
            cell_W = int(cell_P - N_y);
            cell_E = int(cell_P + N_y);
            
            #apply 0-gradient boundary condition
            if cell_P == u_vel_corners[0]: #S-W corner
                u_vel[0, cell_W] = u_vel[0, cell_P];
            elif cell_P == u_vel_corners[1]: #N-W corner
                u_vel[0, cell_W] = u_vel[0, cell_P];
            elif cell_P == u_vel_corners[2]: # S-E corner
                u_vel[0, cell_E] = u_vel[0, cell_P];
            elif cell_P == u_vel_corners[3]: #N-E corner
                u_vel[0, cell_E] = u_vel[0, cell_P];
            elif cell_P in inlet_u_vel_cells:
                u_vel[0, cell_W] = u_vel[0, cell_P];
            elif cell_P in outlet_u_vel_cells:
                u_vel[0, cell_E] = u_vel[0, cell_P];
            #0 velocity at wall is already fully accounted for in the u_vel_sol_mat creation algorithm
            
            u_cell_location = u_cell_location + 1;
            
    #v-velocity corrected velocities + application of zero gradient boundary
    #apply the correction only to the cells strictly in the interior
    #recall from the v_sol_mat creation
    v_cell_location = 0; #this will provide the location of the cell in the solution vector!
    for a_x in range(1, N_x-1):
        for a_y in range(1, N_y):
            cell_P = int(a_x*(N_y+1) + a_y);
            
            #v-vel correction
            v_vel[0, cell_P] = solution_v[0, v_cell_location];
            #set the pressure correction at the walls = 0
            #the v-vel at the wall is 0
            
            cell_W = int(cell_P - (N_y+1));
            cell_E = int(cell_P + (N_y+1));
            
            #apply 0-gradient boundary condition
            if cell_P == v_vel_corners[0]: #S-W corner
                v_vel[0, cell_W] = v_vel[0, cell_P];
            elif cell_P == v_vel_corners[1]: #N-W corner
                v_vel[0, cell_W] = v_vel[0, cell_P];
            elif cell_P == v_vel_corners[2]: # S-E corner
                v_vel[0, cell_E] = v_vel[0, cell_P];
            elif cell_P == v_vel_corners[3]: #N-E corner
                v_vel[0, cell_E] = v_vel[0, cell_P];
            elif cell_P in inlet_v_vel_cells:
                v_vel[0, cell_W] = v_vel[0, cell_P];
            elif cell_P in outlet_v_vel_cells:
                v_vel[0, cell_E] = v_vel[0, cell_P];
            #don't need to set the v-vel 0 at the walls because the solution algorithm acts only on the active cells
            #therefore the wall cells are left untouched
            
            v_cell_location = v_cell_location + 1;
    
    #solve for the pressure correction
    pressure_sol_mat, pressure_source = update_pressure_sol_mat(pressure_sol_mat, pressure_source, N_x, N_y, u_vel_sol_mat,
                           v_vel_sol_mat, u_vel, v_vel, pressure_corners, inlet_pressure_cells,
                           outlet_pressure_cells, bottom_pressure_wall, top_pressure_wall);
    print("pressure source", pressure_source)
    
    pressure_correction = CGS_solver(pressure_sol_mat, pressure_source, pressure_correction);
    print("pressure correction: ", pressure_correction)
    
    #CORRECTION
    
    #pressure correction
    for a in range(0, (N_x - 2)*N_y):
        pressure_cell_location = int(a + N_y);
        pressure_correction_trans[0, pressure_cell_location] = pressure_correction[0, a];
        
    #u-velocity corrected velocities + application of zero gradient boundary
    #apply the correction only to the cells strictly in the interior
    #recall from the u_sol_mat creation
    u_cell_location = 0; #this will provide the location of the cell in the solution vector!
    for a_x in range(1, N_x):
        for a_y in range(0, N_y):
            
            cell_P = int(a_x*N_y + a_y);
        
            location_sol_mat = u_cell_location*5;
            P = int(location_sol_mat + 2);
        
            #location of pressure correction cells
            p_W = int(cell_P - N_y);
            p_E = cell_P;
            
            #u-vel correction
            u_vel[0, cell_P] = solution_u[0, u_cell_location] + (1/u_vel_sol_mat[P, 2])*(pressure_correction_trans[0, p_W] - pressure_correction_trans[0, p_E]);
            
            cell_W = int(cell_P - N_y);
            cell_E = int(cell_P + N_y);
            
            #apply 0-gradient boundary condition
            if cell_P == u_vel_corners[0]: #S-W corner
                u_vel[0, cell_W] = u_vel[0, cell_P];
            elif cell_P == u_vel_corners[1]: #N-W corner
                u_vel[0, cell_W] = u_vel[0, cell_P];
            elif cell_P == u_vel_corners[2]: # S-E corner
                u_vel[0, cell_E] = u_vel[0, cell_P];
            elif cell_P == u_vel_corners[3]: #N-E corner
                u_vel[0, cell_E] = u_vel[0, cell_P];
            elif cell_P in inlet_u_vel_cells:
                u_vel[0, cell_W] = u_vel[0, cell_P];
            elif cell_P in outlet_u_vel_cells:
                u_vel[0, cell_E] = u_vel[0, cell_P];
            #0 velocity at wall is already fully accounted for in the u_vel_sol_mat creation algorithm
            
            u_cell_location = u_cell_location + 1;
    
    #v-velocity corrected velocities + application of zero gradient boundary
    #apply the correction only to the cells strictly in the interior
    #recall from the v_sol_mat creation
    v_cell_location = 0; #this will provide the location of the cell in the solution vector!
    for a_x in range(1, N_x-1):
        for a_y in range(1, N_y):
            cell_P = int(a_x*(N_y+1) + a_y);

            location_sol_mat = v_cell_location*5;
            P = int(location_sol_mat + 2)

            #location of pressure cells
            p_N = int(cell_P - a_x);
            p_S = int(cell_P - a_x - 1);
            
            #v-vel correction
            v_vel[0, cell_P] = solution_v[0, v_cell_location] + (1/v_vel_sol_mat[P, 2])*(pressure_correction_trans[0, p_S] - pressure_correction_trans[0, p_N]);
            #set the pressure correction at the walls = 0
            #the v-vel at the wall is 0
            
            cell_W = int(cell_P - (N_y+1));
            cell_E = int(cell_P + (N_y+1));
            
            #apply 0-gradient boundary condition
            if cell_P == v_vel_corners[0]: #S-W corner
                v_vel[0, cell_W] = v_vel[0, cell_P];
            elif cell_P == v_vel_corners[1]: #N-W corner
                v_vel[0, cell_W] = v_vel[0, cell_P];
            elif cell_P == v_vel_corners[2]: # S-E corner
                v_vel[0, cell_E] = v_vel[0, cell_P];
            elif cell_P == v_vel_corners[3]: #N-E corner
                v_vel[0, cell_E] = v_vel[0, cell_P];
            elif cell_P in inlet_v_vel_cells:
                v_vel[0, cell_W] = v_vel[0, cell_P];
            elif cell_P in outlet_v_vel_cells:
                v_vel[0, cell_E] = v_vel[0, cell_P];
            #don't need to set the v-vel 0 at the walls because the solution algorithm acts only on the active cells
            #therefore the wall cells are left untouched
            
            v_cell_location = v_cell_location + 1;
            
    #under_relaxation
    pressure = pressure_n + under_relax*pressure_correction_trans;
    u_vel = under_relax*u_vel + (1 - under_relax)*u_vel_n;
    v_vel = under_relax*v_vel + (1 - under_relax)*v_vel_n;
    
    #calculate global residual for the velocities
    #we don't have a global residual for the pressure since we don't have a pressure equation, but only a pressure
    #correction equation
    #use the same approach used in GS
    #don't need the boundary cells since they don't enter the calculations and so they don't carry any error
    
    #global residual of u-vel
    for m in range((N_x-1)*N_y):
        S = m*5;
        W = S + 1;
        P = W + 1;
        E = P + 1;
        N = E + 1;
        
        s_1_u = int(u_vel_sol_mat[S, 1]);
        t_1_u = u_vel_sol_mat[S, 2]*u_vel[0, s_1_u];
        w_2_u = int(u_vel_sol_mat[W, 1]);
        t_2_u = u_vel_sol_mat[W, 2]*u_vel[0, w_2_u];
        p_3_u = int(u_vel_sol_mat[P, 1]) #same, it is the diagonal
        t_3_u = u_vel_sol_mat[P, 2]*u_vel[0, p_3_u];
        e_4_u = int(u_vel_sol_mat[E, 1]);
        t_4_u = u_vel_sol_mat[E, 2]*u_vel[0, e_4_u];
        n_5_u = int(u_vel_sol_mat[N, 1]);
        t_5_u = u_vel_sol_mat[N, 2]*u_vel[0, n_5_u];
        
        t_source = u_vel_source[0, m];
        
        residual_u = residual_u + (t_1_u + t_2_u + t_3_u + t_4_u + t_5_u - t_source)**2;
    
    R_G_u = np.sqrt(residual_u);
    
    #global residual of v-vel
    for n in range((N_y-1)*(N_x-2)):
        S = n*5;
        W = S + 1;
        P = W + 1;
        E = P + 1;
        N = E + 1;
        
        s_1_v = int(v_vel_sol_mat[S, 1]);
        t_1_v = v_vel_sol_mat[S, 2]*v_vel[0, s_1_v];
        w_2_v = int(v_vel_sol_mat[W, 1]);
        t_2_v = v_vel_sol_mat[W, 2]*v_vel[0, w_2_v];
        p_3_v = int(v_vel_sol_mat[P, 1]) #same, it is the diagonal
        t_3_v = v_vel_sol_mat[P, 2]*v_vel[0, p_3_v];
        e_4_v = int(v_vel_sol_mat[E, 1]);
        t_4_v = v_vel_sol_mat[E, 2]*v_vel[0, e_4_v];
        n_5_v = int(v_vel_sol_mat[N, 1]);
        t_5_v = v_vel_sol_mat[N, 2]*v_vel[0, n_5_v];
        
        t_source = v_vel_source[0, n];
        
        residual_v = residual_v + (t_1_v + t_2_v + t_3_v + t_4_v + t_5_v - t_source)**2;
    
    R_G_v = np.sqrt(residual_v);

In [ ]:
#plotting the results
#we must define all quantities at the cell centre in order to plot a streamplot for the velocity, and a contour plot
#for the perssure field
#the assignment is performed directly to the mesh-grid of velocity-fields or pressure fields

import matplotlib.pyplot as plt

#cell centre coordinates
X = np.arange(del_l/2, L_x + del_l/2, del_l);
Y = np.arange(del_l/2, L_y + del_l/2, del_l);

#define the velocity vectors
u_coor, v_coor = np.meshgrid(X, Y);

u_coor[:, :] = 0;
v_coor[:, :] = 0;

#assign values to u
for a_x in range(0, N_x):
    for a_y in range(0, N_y):
        west = int(a_x*N_y + a_y);
        east = int(west + N_y);
        
        u_value = (u_vel[0, west] + u_vel[0, east])/2;
        
        u_coor[a_y, a_x] = u_value;

#assign values to v
for a_x in range(0, N_x):
    for a_y in range(0, N_y):
        south = int(a_x*(N_y + 1) + a_y);
        north = int(south + 1);
        
        v_value = (v_vel[0, south] + v_vel[0, north])/2;
        
        v_coor[a_y, a_x] = v_value;
        
#define pressure field
pressure_field = np.zeros((N_y, N_x));

#assign values to p
for a_x in range(0, N_x):
    for a_y in range(0, N_y):
        location = int(a_x*N_y + a_y);
        
        pressure_field[a_y, a_x] = pressure[0, location];
        
#define figure and axes
fig, axes  = plt.subplots(1, 2, figsize = (12, 4));

gr_vel = axes[0];
gr_pressure = axes[1];

gr_vel.streamplot(X, Y, u_coor, v_coor, linewidth=1, density=1, arrowstyle='->', arrowsize=1.5);
gr_vel.set_xlabel("x");
gr_vel.set_ylabel("y");
gr_vel.set_title("Plot of the velocity");
        
pressure_distr = gr_pressure.contourf(X, Y, pressure_field);
plt.colorbar(pressure_distr, ax = gr_pressure);
gr_pressure.set_xlabel("x");
gr_pressure.set_ylabel("y");
gr_pressure.set_title("pressure distribution");

plt.savefig("constant_pressure_pL=10_pR=0.jpeg");

df1 = pd.DataFrame(pressure_field);
df1.to_csv("pressure_fiel_pL=1000_pR=500.csv");

df2 = pd.DataFrame(u_coor)
df2.to_csv("u_velocity_pL=1000_pR=500.csv");

df3 = pd.DataFrame(v_coor);
df3.to_csv("v_velocity_pL=1000_pR=500.csv")